In [ ]:
import astropy.units as u
from astropy.table import Table
from astropy import table
from astropy.coordinates import SkyCoord, Distance
from astropy.time import Time

from astroquery.simbad import Simbad

In [ ]:
names = Table.read('targetnames.txt', format='ascii.no_header', delimiter='|', names=['target'])

In [ ]:
TargetSimbad = Simbad()
TargetSimbad.add_votable_fields('propermotions', 'distance', 'parallax', 'flux(B)', 'flux(V)', 'rv_value', 'sp')
TargetSimbad.add_votable_fields('ra(2;A;ICRS;J2000;2000)', 'dec(2;D;ICRS;J2000;2000)')

In [ ]:
targets = TargetSimbad.query_objects(names['target'])

In [ ]:
targets = table.hstack([names, targets], join_type='exact')

In [ ]:
targets['Distance_distance'].unit = u.pc

In [ ]:
targets['coord'] = SkyCoord(ra=targets['RA_2_A_ICRS_J2000_2000'],
                            dec=targets['DEC_2_D_ICRS_J2000_2000'],
                            unit=(u.hourangle, u.deg),
                            distance=targets['Distance_distance'],
                            pm_ra_cosdec=targets['PMRA'],
                            pm_dec=targets['PMDEC'],
                            frame='icrs', obstime='J2000', equinox='J2000')

In [ ]:
targets.write('targets.fits')